In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from statannotations.Annotator import Annotator
import seaborn as sns
from tqdm import tqdm


In [ ]:
# Путь к папке с образцами
base_dir = "E:/YandexDisk/Work/bbd/fmba/lesnoy_2025/results/tapes_annotations_2025"

sheet_name = "Top 300"
prob_thld = 0.5

In [ ]:
all_results = []

# Проходим по папкам
  
for file in (pbar := tqdm(os.listdir(base_dir))):
    pbar.set_description(f"{file}")
    
    if file.endswith(".vcf.xlsx"):
        file_path = os.path.join(base_dir, file)

        try:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            # Убедимся, что нужные колонки присутствуют
            required_cols = [
                "Ген", "Вероятность патогенности", "Классификация ACMG",
                "Зиготность", "Описание заболевания"
            ]
            if not all(col in df.columns for col in required_cols):
                print(f"Пропущен файл {file_path} — нет нужных колонок.")
                continue
            
            # Фильтрация
            filtered = df[df["Вероятность патогенности"] >= prob_thld].copy()

            if not filtered.empty:
                sample_number = file.split(".")[0]
                filtered["Номер образца"] = sample_number

                all_results.append(filtered[required_cols + ["Номер образца"]])

        except Exception as e:
            print(f"Ошибка при обработке файла {file_path}: {e}")

# Объединяем в один датафрейм
if all_results:
    mutations_df = pd.concat(all_results, ignore_index=True)

    # Переименование колонок
    mutations_df.rename(columns={
        "Ген": "Gene",
        "Вероятность патогенности": "Pathogenicity Probability",
        "Классификация ACMG": "ACMG Classification",
        "Зиготность": "Zygosity",
        "Описание заболевания": "Disease Description",
        "Номер образца": "Sample"
    }, inplace=True)

    # Сохраняем при необходимости:
    # mutations_df.to_excel("filtered_mutations.xlsx", index=False)
    # mutations_df.to_csv("filtered_mutations.csv", index=False)

    display(mutations_df.head())
else:
    print("Подходящих данных не найдено.")


In [ ]:
mutations_df.to_excel(f"E:/YandexDisk/Work/bbd/fmba/lesnoy_2025/results/filtered_mutations_{sheet_name}_{prob_thld}.xlsx", index=False)

In [ ]:
mutations_df = pd.read_excel(f"E:/YandexDisk/Work/bbd/fmba/lesnoy_2025/results/filtered_mutations_{sheet_name}_{prob_thld}.xlsx")
mutations_df["number"] = mutations_df["Sample"].astype(str)
samples = mutations_df['number'].unique().tolist()
print(len(samples))

In [ ]:
# Чтение файла
patient_groups_df = pd.read_excel(f"E:/YandexDisk/Work/bbd/fmba/lesnoy_2025/Таблица2025.xlsx")
patient_groups_df["number"] = patient_groups_df["работник_ID"].astype(str)
patient_groups_df = patient_groups_df[patient_groups_df["number"].isin(samples)]

# Просмотр первых строк
display(patient_groups_df.head())

In [ ]:
# Переименование колонки
patient_groups_df.rename(columns={"признак 500 здоровые больные": "health_status"}, inplace=True)

# Замена значений
patient_groups_df["health_status"] = patient_groups_df["health_status"].replace({
    "536_б": "diseased",
    "536_з": "healthy"
})

# Проверим результат
display(patient_groups_df["health_status"].value_counts())


In [ ]:
patient_groups_df

In [ ]:
# Объединяем по колонке number
merged_df = mutations_df.merge(patient_groups_df, on="number")

# Среднее число мутаций на пациента в каждой группе
mean_counts = merged_df.groupby("health_status")["Gene"].count() / merged_df.groupby("health_status")["Sample"].nunique()

In [ ]:
merged_df

In [ ]:
#merged_df.to_excel("merged_df_data.xlsx", index=False)

In [ ]:
# Подготовка данных
grouped = merged_df.groupby(["health_status", "Sample"])["Gene"].count().reset_index(name="mutation_count")

# Группы для сравнения
pairs = [("healthy", "diseased")]

plt.figure(figsize=(6, 5))
ax = sns.barplot(
    data=grouped,
    x="health_status",
    y="mutation_count",
    estimator="mean",
    ci="sd",
    capsize=0.2,
    errcolor='black',
    errwidth=1.5,
    palette="Set2"
)

# Добавление статистической аннотации
annotator = Annotator(ax, pairs, data=grouped, x="health_status", y="mutation_count")
annotator.configure(test='t-test_ind', text_format='star', loc='inside', comparisons_correction=None)
annotator.apply_and_annotate()

plt.ylabel("Среднее число мутаций")
plt.xlabel("Группа")
plt.title("Среднее количество мутаций на пациента")
plt.tight_layout()
# plt.savefig("среднее число мутаций.pdf", format="pdf", dpi=300)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Группировка мутаций по пациентам и классификации ACMG
grouped_acmg = merged_df.groupby(["health_status", "ACMG Classification", "Sample"])["Gene"].count().reset_index(name="mutation_count")

# Визуализация
plt.figure(figsize=(8, 6))
ax = sns.barplot(
    data=grouped_acmg,
    x="health_status",
    y="mutation_count",
    hue="ACMG Classification",  # Добавляем классификацию ACMG
    estimator="mean",  # Среднее значение
    errorbar="sd",  # Ошибки по стандартному отклонению
    capsize=0.2,
    palette="Set2",
    err_kws={'color': 'black', 'linewidth': 1.5}  # Ошибки с черным цветом и шириной 1.5
)

# Подписи и заголовок
plt.ylabel("Среднее количество мутаций на пациента")
plt.xlabel("Группа")
plt.title("Среднее количество мутаций на пациента (по классификации ACMG)")

# Сохранение диаграммы в PDF
plt.tight_layout()
# plt.savefig("avg_mutations_per_patient_with_acmg.pdf", format="pdf", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Считаем количество мутаций для каждого гена
gene_counts = merged_df["Gene"].value_counts()

# Выбираем топ 50 генов с наибольшей частотой мутаций
top_50_genes = gene_counts.head(50)

# Визуализируем
plt.figure(figsize=(10, 6))
sns.barplot(x=top_50_genes.index, y=top_50_genes.values, hue=None, palette="Blues_d")

# Настройка графика
plt.title("Топ 50 генов по частоте мутаций")
plt.xlabel("Ген")
plt.ylabel("Частота мутаций")
plt.xticks(rotation=90)  # Поворот меток по оси X для удобства чтения
plt.tight_layout()

# Сохраняем график
# plt.savefig("top_50_mutation_genes.pdf", format="pdf", dpi=300)

# Показываем график
plt.show()


In [ ]:
# Подсчитываем количество мутаций для каждого гена в обеих группах
mutation_counts = merged_df.groupby(["health_status", "Gene"]).size().unstack(fill_value=0)

# Считаем общее количество мутаций для каждого гена (по обеим группам)
total_mutation_counts = mutation_counts.sum(axis=0)

# Выбираем топ-50 генов с наибольшим количеством мутаций
top_50_genes = total_mutation_counts.nlargest(50).index

# Отбираем данные для этих топ-50 генов
mutation_counts_filtered = mutation_counts.loc[:, top_50_genes]

# Визуализируем heatmap для этих 50 генов
plt.figure(figsize=(12, 8))
sns.heatmap(mutation_counts_filtered.T, cmap="Blues", annot=True, fmt="d", cbar_kws={'label': 'Частота мутаций'}, linewidths=0.5)

# Настройки графика
plt.title("Частота геномных вариантов по 50 наиболее часто мутирующим генам для здоровых и больных")
plt.xlabel("Группы (здоровые и больные)")
plt.ylabel("Гены")
plt.xticks(rotation=45)  # Поворот меток по оси X для удобства чтения
plt.tight_layout()

# Сохраняем график в PDF
# plt.savefig("mutation_frequency_heatmap_top_50_genes.pdf", format="pdf", dpi=300)

# Показываем график
plt.show()


In [ ]:
top_genes = merged_df['Gene'].value_counts().head(50).index
df_top = merged_df[merged_df['Gene'].isin(top_genes)]


In [ ]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2
import matplotlib as mpl
import seaborn as sns

# Настройка стиля — приближённый к Nature
sns.set_theme(style="white")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'figure.dpi': 300,
    'savefig.dpi': 600,
    'figure.figsize': (6, 6)
})

# Получаем множества генов для каждой группы
genes_healthy = set(merged_df[merged_df['health_status'] == 'healthy']['Gene'].dropna())
genes_diseased = set(merged_df[merged_df['health_status'] == 'diseased']['Gene'].dropna())

# Построение диаграммы Венна
plt.figure(figsize=(6, 6))
venn = venn2([genes_healthy, genes_diseased],
             set_labels=('Healthy', 'Diseased'),
             set_colors=('skyblue', 'salmon'),
             alpha=0.8)

# Улучшение внешнего вида шрифтов и заголовка
plt.title("Shared and Unique Genes Between Groups", fontsize=16, weight='bold', pad=20)
for text in venn.set_labels:
    text.set_fontsize(12)
for text in venn.subset_labels:
    if text: text.set_fontsize(14)

sns.despine()
plt.tight_layout()

# Сохранение в PDF высокого качества
# plt.savefig("venn_genes_healthy_vs_diseased_nature.pdf", format='pdf', bbox_inches='tight')
plt.show()

# Печать статистики
print(f"Гены только у healthy: {len(genes_healthy - genes_diseased)}")
print(f"Гены только у diseased: {len(genes_diseased - genes_healthy)}")
print(f"Общие гены: {len(genes_healthy & genes_diseased)}")



In [ ]:
import requests
import json
import pandas as pd

# Функция для отправки списка генов на Enrichr и получения результатов
def enrichr_enrichment(genes):
    url = 'https://maayanlab.cloud/Enrichr/addList'
    response = requests.post(url, files={'file': ('genes.txt', '\n'.join(genes))})
    if response.status_code == 200:
        list_id = response.json()['userListId']
        enrichment_url = f'https://maayanlab.cloud/Enrichr/enrich?userListId={list_id}&backgroundType=KEGG_2016'
        enrichment_response = requests.get(enrichment_url)
        return enrichment_response.json()
    else:
        print("Ошибка при отправке данных на Enrichr")
        return None

# Функция для получения обогащения
def get_enrichment_for_genes(genes):
    enrichment_results = enrichr_enrichment(genes)
    if enrichment_results:
        # Проверим, что именно вернул API
        print(json.dumps(enrichment_results, indent=4))  # Печать для проверки структуры данных
        # Преобразуем результат в DataFrame для удобного отображения
        if 'KEGG_2016' in enrichment_results:
            enrichment_data = enrichment_results['KEGG_2016']
            # Определим количество колонок, чтобы правильно создать DataFrame
            num_columns = len(enrichment_data[0])
            column_names = ['Rank', 'Term', 'P-value', 'Z-score', 'Combined Score', 'Genes', 'Adjusted P-value']
            
            if num_columns > len(column_names):
                # Если количество колонок больше, добавляем дополнительные колонки
                column_names += [f'Extra_{i}' for i in range(len(column_names), num_columns)]
            
            # Создаем DataFrame с нужными колонками
            enrichment_df = pd.DataFrame(enrichment_data, columns=column_names)
            return enrichment_df
        else:
            print("Нет данных по KEGG_2016.")
            return None
    else:
        return None

# Разделяем гены по статусу здоровья
genes_healthy = set(merged_df[merged_df['health_status'] == 'healthy']['Gene'])
genes_diseased = set(merged_df[merged_df['health_status'] == 'diseased']['Gene'])

# Находим уникальные и общие гены
unique_healthy = genes_healthy - genes_diseased
unique_diseased = genes_diseased - genes_healthy
common_genes = genes_healthy & genes_diseased

# Выводим количество генов для каждой группы
print(f'Уникальные гены для здоровых: {len(unique_healthy)}')
print(f'Уникальные гены для больных: {len(unique_diseased)}')
print(f'Общие гены: {len(common_genes)}')

# Получаем обогащение для уникальных и общих генов
healthy_enrichment = get_enrichment_for_genes(unique_healthy)
diseased_enrichment = get_enrichment_for_genes(unique_diseased)
common_enrichment = get_enrichment_for_genes(common_genes)

# Выводим результаты обогащения и сохраняем в файлы
if healthy_enrichment is not None:
    print("\nОбогащение для уникальных генов (здоровые):")
    print(healthy_enrichment.head())
    # Сохраняем в файл Excel
    healthy_enrichment.to_excel('healthy_enrichment.xlsx', index=False)

if diseased_enrichment is not None:
    print("\nОбогащение для уникальных генов (больные):")
    print(diseased_enrichment.head())
    # Сохраняем в файл Excel
    diseased_enrichment.to_excel('diseased_enrichment.xlsx', index=False)

if common_enrichment is not None:
    print("\nОбогащение для общих генов:")
    print(common_enrichment.head())
    # Сохраняем в файл Excel
    common_enrichment.to_excel('common_enrichment.xlsx', index=False)


In [ ]:
import requests
import json
import pandas as pd

# Функция для отправки списка генов на Enrichr и получения результатов
def enrichr_enrichment(genes):
    url = 'https://maayanlab.cloud/Enrichr/addList'
    response = requests.post(url, files={'file': ('genes.txt', '\n'.join(genes))})
    if response.status_code == 200:
        list_id = response.json()['userListId']
        enrichment_url = f'https://maayanlab.cloud/Enrichr/enrich?userListId={list_id}&backgroundType=Jensen_DISEASES'
        enrichment_response = requests.get(enrichment_url)
        return enrichment_response.json()
    else:
        print("Ошибка при отправке данных на Enrichr")
        return None

# Функция для получения обогащения
def get_enrichment_for_genes(genes):
    enrichment_results = enrichr_enrichment(genes)
    if enrichment_results:
        # Проверим, что именно вернул API
        print(json.dumps(enrichment_results, indent=4))  # Печать для проверки структуры данных
        # Преобразуем результат в DataFrame для удобного отображения
        if 'Jensen_DISEASES' in enrichment_results:
            enrichment_data = enrichment_results['Jensen_DISEASES']
            # Определим количество колонок, чтобы правильно создать DataFrame
            num_columns = len(enrichment_data[0])
            column_names = ['Rank', 'Term', 'P-value', 'Z-score', 'Combined Score', 'Genes', 'Adjusted P-value']
            
            if num_columns > len(column_names):
                # Если количество колонок больше, добавляем дополнительные колонки
                column_names += [f'Extra_{i}' for i in range(len(column_names), num_columns)]
            
            # Создаем DataFrame с нужными колонками
            enrichment_df = pd.DataFrame(enrichment_data, columns=column_names)
            return enrichment_df
        else:
            print("Нет данных по Jensen_DISEASES.")
            return None
    else:
        return None

# Разделяем гены по статусу здоровья
genes_healthy = set(merged_df[merged_df['health_status'] == 'healthy']['Gene'])
genes_diseased = set(merged_df[merged_df['health_status'] == 'diseased']['Gene'])

# Находим уникальные и общие гены
unique_healthy = genes_healthy - genes_diseased
unique_diseased = genes_diseased - genes_healthy
common_genes = genes_healthy & genes_diseased

# Выводим количество генов для каждой группы
print(f'Уникальные гены для здоровых: {len(unique_healthy)}')
print(f'Уникальные гены для больных: {len(unique_diseased)}')
print(f'Общие гены: {len(common_genes)}')

# Получаем обогащение для уникальных и общих генов
healthy_enrichment = get_enrichment_for_genes(unique_healthy)
diseased_enrichment = get_enrichment_for_genes(unique_diseased)
common_enrichment = get_enrichment_for_genes(common_genes)

# Выводим результаты обогащения и сохраняем в файлы
if healthy_enrichment is not None:
    print("\nОбогащение для уникальных генов (здоровые):")
    print(healthy_enrichment.head())
    # Сохраняем в файл Excel
    healthy_enrichment.to_excel('healthy_enrichment_Jensen_DISEASES.xlsx', index=False)

if diseased_enrichment is not None:
    print("\nОбогащение для уникальных генов (больные):")
    print(diseased_enrichment.head())
    # Сохраняем в файл Excel
    diseased_enrichment.to_excel('diseased_enrichment_Jensen_DISEASES.xlsx', index=False)

if common_enrichment is not None:
    print("\nОбогащение для общих генов:")
    print(common_enrichment.head())
    # Сохраняем в файл Excel
    common_enrichment.to_excel('common_enrichment_Jensen_DISEASES.xlsx', index=False)


In [ ]:
healthy_enrichment 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Фильтрация по Adjusted P-value < 0.05 для каждого из DataFrame
healthy_enrichment_filtered = healthy_enrichment[healthy_enrichment['Adjusted P-value'] < 0.08]
diseased_enrichment_filtered = diseased_enrichment[diseased_enrichment['Adjusted P-value'] < 0.08]
common_enrichment_filtered = common_enrichment[common_enrichment['Adjusted P-value'] < 0.08]

# Извлекаем только термины и их комбинированные оценки для каждого набора данных
healthy_terms = set(healthy_enrichment_filtered['Term'])
diseased_terms = set(diseased_enrichment_filtered['Term'])
common_terms = set(common_enrichment_filtered['Term'])

# Создаем словарь для термина и его присутствия в разных группах
all_terms = list(healthy_terms.union(diseased_terms, common_terms))
term_presence = {
    'Healthy': [1 if term in healthy_terms else 0 for term in all_terms],
    'Diseased': [1 if term in diseased_terms else 0 for term in all_terms],
    'Common': [1 if term in common_terms else 0 for term in all_terms]
}

# Преобразуем в DataFrame
term_presence_df = pd.DataFrame(term_presence, index=all_terms)

# Визуализируем heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(term_presence_df, cmap='Blues', annot=True, cbar=False, linewidths=0.5)
plt.title('Представленность заболеваний в группах (Adjusted P-value < 0.05)', fontsize=16)
plt.xlabel('Группы', fontsize=14)
plt.ylabel('Заболевания', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Настройка стиля — приближённый к Nature
sns.set_theme(style="whitegrid")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'figure.figsize': (10, 12)
})

# Фильтрация по Adjusted P-value < 0.05
healthy_enrichment_filtered = healthy_enrichment[healthy_enrichment['Adjusted P-value'] < 0.09]
diseased_enrichment_filtered = diseased_enrichment[diseased_enrichment['Adjusted P-value'] < 0.09]
common_enrichment_filtered = common_enrichment[common_enrichment['Adjusted P-value'] < 0.09]

# Извлекаем термины
healthy_terms = set(healthy_enrichment_filtered['Term'])
diseased_terms = set(diseased_enrichment_filtered['Term'])
common_terms = set(common_enrichment_filtered['Term'])

# Все уникальные термины
all_terms = sorted(list(healthy_terms | diseased_terms | common_terms))

# Создаем таблицу присутствия
term_matrix = pd.DataFrame({
    'Healthy': [1 if term in healthy_terms else 0 for term in all_terms],
    'Diseased': [1 if term in diseased_terms else 0 for term in all_terms],
    'Common': [1 if term in common_terms else 0 for term in all_terms]
}, index=all_terms)

# Цвета и маска
cmap = sns.color_palette("crest", as_cmap=True)

# Построение heatmap
plt.figure(figsize=(10, len(all_terms)*0.3 + 2))
ax = sns.heatmap(term_matrix,
                 cmap=cmap,
                 linewidths=0.5,
                 linecolor='white',
                 annot=True,
                 fmt='d',
                 cbar=False)

# Оформление
plt.title('Enrichment of Disease-Associated Terms\n(Jensen DISEASES, FDR < 0.09)', fontsize=16, weight='bold', pad=20)
plt.xlabel('Group', fontsize=14)
plt.ylabel('Disease Term', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(fontsize=10)
sns.despine(left=True, bottom=True)
plt.tight_layout()

# Сохранение в файл высокого качества
plt.savefig('enrichment_heatmap_nature_style.png', dpi=600)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Настройка стиля — приближённый к Nature
sns.set_theme(style="whitegrid")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'figure.figsize': (10, 12)
})

# Фильтрация по Adjusted P-value < 0.05
healthy_enrichment_filtered = healthy_enrichment[healthy_enrichment['Adjusted P-value'] < 0.09]
diseased_enrichment_filtered = diseased_enrichment[diseased_enrichment['Adjusted P-value'] < 0.09]

# Удаляем общие термины
healthy_terms = set(healthy_enrichment_filtered['Term'])
diseased_terms = set(diseased_enrichment_filtered['Term'])
unique_healthy_terms = healthy_terms - diseased_terms
unique_diseased_terms = diseased_terms - healthy_terms

# Все уникальные термины
all_unique_terms = sorted(list(unique_healthy_terms | unique_diseased_terms))

# Создаем таблицу присутствия (без колонки 'Common')
term_matrix = pd.DataFrame({
    'Healthy': [1 if term in unique_healthy_terms else 0 for term in all_unique_terms],
    'Diseased': [1 if term in unique_diseased_terms else 0 for term in all_unique_terms]
}, index=all_unique_terms)

# Цветовая палитра
cmap = sns.color_palette("crest", as_cmap=True)

# Построение heatmap
plt.figure(figsize=(10, len(all_unique_terms)*0.3 + 2))
ax = sns.heatmap(term_matrix,
                 cmap=cmap,
                 linewidths=0.5,
                 linecolor='white',
                 annot=True,
                 fmt='d',
                 cbar=False)

# Оформление
plt.title('Unique Disease-Associated Terms Enrichment\n(Jensen DISEASES, FDR < 0.09)', fontsize=16, weight='bold', pad=20)
plt.xlabel('Group', fontsize=14)
plt.ylabel('Disease Term', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(fontsize=10)
sns.despine(left=True, bottom=True)
plt.tight_layout()

# Сохранение в файл
plt.savefig('enrichment_heatmap_unique_terms_nature_style.png', dpi=600)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Ключевые слова, связанные с липидами
lipid_keywords = ['lipid', 'lipo', 'cholesterol', 'fatty', 'triglyceride', 'adipose', 'lipoprotein', 'obesity']

# Фильтрация по наличию ключевых слов в названиях термина (без учёта регистра)
def filter_lipid_terms(df):
    return df[df['Term'].str.lower().str.contains('|'.join(lipid_keywords))]

# Фильтруем обогащение по ключевым словам
healthy_lipid = filter_lipid_terms(healthy_enrichment)
diseased_lipid = filter_lipid_terms(diseased_enrichment)

# Убираем пересекающиеся термины
healthy_terms = set(healthy_lipid['Term'])
diseased_terms = set(diseased_lipid['Term'])
unique_healthy_terms = healthy_terms - diseased_terms
unique_diseased_terms = diseased_terms - healthy_terms

# Все уникальные термины
all_unique_terms = sorted(list(unique_healthy_terms | unique_diseased_terms))

# Создаем таблицу присутствия
term_matrix = pd.DataFrame({
    'Healthy': [1 if term in unique_healthy_terms else 0 for term in all_unique_terms],
    'Diseased': [1 if term in unique_diseased_terms else 0 for term in all_unique_terms]
}, index=all_unique_terms)

# Настройка графики в стиле Nature
sns.set_theme(style="whitegrid")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'figure.figsize': (10, max(6, len(all_unique_terms)*0.3 + 2))
})

# Цветовая палитра
cmap = sns.color_palette("flare", as_cmap=True)

# Построение heatmap
plt.figure()
ax = sns.heatmap(term_matrix,
                 cmap=cmap,
                 linewidths=0.5,
                 linecolor='white',
                 annot=True,
                 fmt='d',
                 cbar=False)

# Оформление
plt.title('Lipid-Associated Disease Terms Enrichment\n(Jensen DISEASES)', fontsize=16, weight='bold', pad=20)
plt.xlabel('Group', fontsize=14)
plt.ylabel('Disease Term', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(fontsize=10)
sns.despine(left=True, bottom=True)
plt.tight_layout()

# Сохраняем картинку
plt.savefig('lipid_term_enrichment_heatmap.png', dpi=600)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Ключевые слова, связанные с раком
cancer_keywords = ['cancer', 'tumor', 'carcinoma', 'neoplasm', 'oncogenesis', 'malignancy', 'metastasis', 'sarcoma']

# Фильтрация по наличию ключевых слов в названиях термина (без учёта регистра)
def filter_cancer_terms(df):
    return df[df['Term'].str.lower().str.contains('|'.join(cancer_keywords))]

# Фильтруем обогащение по ключевым словам
healthy_cancer = filter_cancer_terms(healthy_enrichment)
diseased_cancer = filter_cancer_terms(diseased_enrichment)

# Убираем пересекающиеся термины
healthy_terms_cancer = set(healthy_cancer['Term'])
diseased_terms_cancer = set(diseased_cancer['Term'])
unique_healthy_terms_cancer = healthy_terms_cancer - diseased_terms_cancer
unique_diseased_terms_cancer = diseased_terms_cancer - healthy_terms_cancer

# Все уникальные термины
all_unique_terms_cancer = sorted(list(unique_healthy_terms_cancer | unique_diseased_terms_cancer))

# Создаем таблицу присутствия
term_matrix_cancer = pd.DataFrame({
    'Healthy': [1 if term in unique_healthy_terms_cancer else 0 for term in all_unique_terms_cancer],
    'Diseased': [1 if term in unique_diseased_terms_cancer else 0 for term in all_unique_terms_cancer]
}, index=all_unique_terms_cancer)

# Настройка графики в стиле Nature
sns.set_theme(style="whitegrid")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'figure.figsize': (10, max(6, len(all_unique_terms_cancer)*0.3 + 2))
})

# Цветовая палитра
cmap = sns.color_palette("viridis", as_cmap=True)

# Построение heatmap
plt.figure()
ax = sns.heatmap(term_matrix_cancer,
                 cmap=cmap,
                 linewidths=0.5,
                 linecolor='white',
                 annot=True,
                 fmt='d',
                 cbar=False)

# Оформление
plt.title('Cancer-Associated Disease Terms Enrichment\n(Jensen DISEASES)', fontsize=16, weight='bold', pad=20)
plt.xlabel('Group', fontsize=14)
plt.ylabel('Disease Term', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(fontsize=10)
sns.despine(left=True, bottom=True)
plt.tight_layout()

# Сохраняем картинку
plt.savefig('cancer_term_enrichment_heatmap.png', dpi=600)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Ключевые слова, связанные с гипертонией
hypertension_keywords = ['Hypertension', 'hypertension', 'High blood pressure', 'hypertensive', 'arterial hypertension', 'blood pressure']

# Фильтрация по наличию ключевых слов в названиях термина (без учёта регистра)
def filter_hypertension_terms(df):
    return df[df['Term'].str.lower().str.contains('|'.join(hypertension_keywords))]

# Фильтруем обогащение по ключевым словам
healthy_hypertension = filter_hypertension_terms(healthy_enrichment)
diseased_hypertension = filter_hypertension_terms(diseased_enrichment)

# Убираем пересекающиеся термины
healthy_terms_hypertension = set(healthy_hypertension['Term'])
diseased_terms_hypertension = set(diseased_hypertension['Term'])
unique_healthy_terms_hypertension = healthy_terms_hypertension - diseased_terms_hypertension
unique_diseased_terms_hypertension = diseased_terms_hypertension - healthy_terms_hypertension

# Все уникальные термины
all_unique_terms_hypertension = sorted(list(unique_healthy_terms_hypertension | unique_diseased_terms_hypertension))

# Создаем таблицу присутствия
term_matrix_hypertension = pd.DataFrame({
    'Healthy': [1 if term in unique_healthy_terms_hypertension else 0 for term in all_unique_terms_hypertension],
    'Diseased': [1 if term in unique_diseased_terms_hypertension else 0 for term in all_unique_terms_hypertension]
}, index=all_unique_terms_hypertension)

# Настройка графики в стиле Nature
sns.set_theme(style="whitegrid")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'figure.figsize': (10, max(6, len(all_unique_terms_hypertension)*0.3 + 2))
})

# Цветовая палитра
cmap = sns.color_palette("viridis", as_cmap=True)

# Построение heatmap
plt.figure()
ax = sns.heatmap(term_matrix_hypertension,
                 cmap=cmap,
                 linewidths=0.5,
                 linecolor='white',
                 annot=True,
                 fmt='d',
                 cbar=False)

# Оформление
plt.title('Hypertension-Associated Disease Terms Enrichment\n(Jensen DISEASES)', fontsize=16, weight='bold', pad=20)
plt.xlabel('Group', fontsize=14)
plt.ylabel('Disease Term', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(fontsize=10)
sns.despine(left=True, bottom=True)
plt.tight_layout()

# Сохраняем картинку
plt.savefig('hypertension_term_enrichment_heatmap.png', dpi=600)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Ключевые слова для заболеваний
cholesterolemia_keywords = ['hypercholesterolemia', 'high cholesterol', 'cholesterol', 'lipid']
obesity_keywords = ['obesity', 'overweight', 'fatty', 'adiposity']
blood_pressure_keywords = ['hypertension', 'high blood pressure', 'hypertensive', 'arterial hypertension', 'blood pressure']
cardiac_disease_keywords = ['cardiovascular', 'heart disease', 'coronary artery disease', 'myocardial infarction', 'stroke', 'cardiac']

# Функция фильтрации по ключевым словам
def filter_terms(df, keywords):
    return df[df['Term'].str.lower().str.contains('|'.join(keywords))]

# Фильтрация по заболеваниям
healthy_cholesterolemia = filter_terms(healthy_enrichment, cholesterolemia_keywords)
diseased_cholesterolemia = filter_terms(diseased_enrichment, cholesterolemia_keywords)

healthy_obesity = filter_terms(healthy_enrichment, obesity_keywords)
diseased_obesity = filter_terms(diseased_enrichment, obesity_keywords)

healthy_blood_pressure = filter_terms(healthy_enrichment, blood_pressure_keywords)
diseased_blood_pressure = filter_terms(diseased_enrichment, blood_pressure_keywords)

healthy_cardiac_disease = filter_terms(healthy_enrichment, cardiac_disease_keywords)
diseased_cardiac_disease = filter_terms(diseased_enrichment, cardiac_disease_keywords)

# Получение уникальных терминов для каждого заболевания
unique_healthy_cholesterolemia, unique_diseased_cholesterolemia = get_unique_terms(healthy_cholesterolemia['Term'], diseased_cholesterolemia['Term'])
unique_healthy_obesity, unique_diseased_obesity = get_unique_terms(healthy_obesity['Term'], diseased_obesity['Term'])
unique_healthy_blood_pressure, unique_diseased_blood_pressure = get_unique_terms(healthy_blood_pressure['Term'], diseased_blood_pressure['Term'])
unique_healthy_cardiac_disease, unique_diseased_cardiac_disease = get_unique_terms(healthy_cardiac_disease['Term'], diseased_cardiac_disease['Term'])

# Объединяем все уникальные термины для построения heatmap
all_terms = sorted(list(
    unique_healthy_cholesterolemia | unique_diseased_cholesterolemia |
    unique_healthy_obesity | unique_diseased_obesity |
    unique_healthy_blood_pressure | unique_diseased_blood_pressure |
    unique_healthy_cardiac_disease | unique_diseased_cardiac_disease
))

# Создаем таблицы присутствия для всех заболеваний
term_matrix = pd.DataFrame({
    'Healthy (Cholesterolemia)': [1 if term in unique_healthy_cholesterolemia else 0 for term in all_terms],
    'Diseased (Cholesterolemia)': [1 if term in unique_diseased_cholesterolemia else 0 for term in all_terms],
    'Healthy (Obesity)': [1 if term in unique_healthy_obesity else 0 for term in all_terms],
    'Diseased (Obesity)': [1 if term in unique_diseased_obesity else 0 for term in all_terms],
    'Healthy (Hypertension)': [1 if term in unique_healthy_blood_pressure else 0 for term in all_terms],
    'Diseased (Hypertension)': [1 if term in unique_diseased_blood_pressure else 0 for term in all_terms],
    'Healthy (Cardiac Disease)': [1 if term in unique_healthy_cardiac_disease else 0 for term in all_terms],
    'Diseased (Cardiac Disease)': [1 if term in unique_diseased_cardiac_disease else 0 for term in all_terms]
}, index=all_terms)

# Настройка графики в стиле Nature
sns.set_theme(style="whitegrid")
mpl.rcParams.update({
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'figure.figsize': (12, 10)
})

# Цветовая палитра
cmap = sns.color_palette("viridis", as_cmap=True)

# Построение heatmap для всех заболеваний
plt.figure(figsize=(14, 10))
ax = sns.heatmap(term_matrix,
                 cmap=cmap,
                 linewidths=0.5,
                 linecolor='white',
                 annot=True,
                 fmt='d',
                 cbar=True)

# Оформление
plt.title('Enrichment of Disease Terms Associated with Cholesterolemia, Obesity, Hypertension, and Cardiac Disease\n(Jensen DISEASES)', fontsize=16, weight='bold', pad=20)
plt.xlabel('Group', fontsize=14)
plt.ylabel('Disease Term', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(fontsize=10)
sns.despine(left=True, bottom=True)
plt.tight_layout()

# Сохраняем картинку
plt.savefig('all_disease_terms_enrichment_heatmap.png', dpi=600)
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Группируем данные по 'Gene' и 'health_status' и считаем количество мутаций для каждого образца
mutation_counts = merged_df.groupby(['Gene', 'health_status', 'Sample']).size().unstack(fill_value=0)

# Проверим, что таблица выглядит правильно
print(mutation_counts.head())

# Нормализация данных для PCA (нормализуем по каждому образцу)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(mutation_counts.T)  # транспонируем таблицу, чтобы строки были образцами

# Применяем PCA для снижения размерности
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Создаем DataFrame для удобства работы с результатами PCA
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

# Добавляем информацию о статусе здоровья и образцах
pca_df['Sample'] = mutation_counts.columns
pca_df['health_status'] = merged_df.groupby('Sample')['health_status'].first().values  # Добавляем статус здоровья каждого образца

# Визуализируем результаты PCA с использованием seaborn
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='health_status', style='health_status', data=pca_df, palette='coolwarm', s=100, edgecolor='k')

# Настройки для графика
plt.title('PCA of Mutation Counts for Healthy and Diseased Samples', fontsize=16)
plt.xlabel(f'Principal Component 1 ({round(pca.explained_variance_ratio_[0] * 100, 2)}%)', fontsize=14)
plt.ylabel(f'Principal Component 2 ({round(pca.explained_variance_ratio_[1] * 100, 2)}%)', fontsize=14)
plt.legend(title='Health Status')
plt.tight_layout()

# Показываем график
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Фильтрация по ACMG Classification: только патогенные и вероятно патогенные варианты
filtered_df = merged_df[merged_df['ACMG Classification'].isin(['Pathogenic', 'Likely Pathogenic'])]

# Группируем данные по 'Gene' и 'health_status' и считаем количество мутаций для каждого образца
mutation_counts = filtered_df.groupby(['Gene', 'health_status', 'Sample']).size().unstack(fill_value=0)

# Проверим, что таблица выглядит правильно
print(mutation_counts.head())

# Нормализация данных для PCA (нормализуем по каждому образцу)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(mutation_counts.T)  # транспонируем таблицу, чтобы строки были образцами

# Применяем PCA для снижения размерности
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Создаем DataFrame для удобства работы с результатами PCA
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

# Добавляем информацию о статусе здоровья и образцах
pca_df['Sample'] = mutation_counts.columns
pca_df['health_status'] = filtered_df.groupby('Sample')['health_status'].first().values  # Добавляем статус здоровья каждого образца

# Визуализируем результаты PCA с использованием seaborn
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='health_status', style='health_status', data=pca_df, palette='coolwarm', s=100, edgecolor='k')

# Настройки для графика
plt.title('PCA of Mutation Counts (Pathogenic & Likely Pathogenic Variants)', fontsize=16)
plt.xlabel(f'Principal Component 1 ({round(pca.explained_variance_ratio_[0] * 100, 2)}%)', fontsize=14)
plt.ylabel(f'Principal Component 2 ({round(pca.explained_variance_ratio_[1] * 100, 2)}%)', fontsize=14)
plt.legend(title='Health Status')
plt.tight_layout()

# Показываем график
plt.show()
